### Workbook for extracting WorldPop population estimates for varying bridge catchment distances
Week of May 12, 2025
Author: Adele Birkenes

In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio.mask import mask
import os
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import GeoJson

Task 1: Read in data on bridge sites, population (WorldPop), and village boundaries

Note: The village boundaries are not used in this analysis, but they contain the custom Rwanda Transverse Mercator projection that will be copied over to the bridge sites and WorldPop data

In [5]:
synced_path = "../../synced-data/population-exploration/"
unsynced_path = "../../unsynced-data"

# Village boundaries (geodataframe)
village_boundaries_fp = os.path.join(synced_path,"Rwanda Village Boundaries/Village.shp")
Rwanda_village_boundaries = gpd.read_file(village_boundaries_fp)

# Bridge sites (dataframe)
bridge_sites_fp = os.path.join(synced_path, "Rwanda Sites with All Population Fields_Exported 2025.04.11.csv")
bridge_sites = pd.read_csv(bridge_sites_fp, encoding='ISO-8859-1') # Note: This encoding accommodates special characters

In [6]:
# Convert bridge sites dataframe to geodataframe that has custom Rwanda TM CRS copied from village boundaries geodataframe
def map_bridges(bridges, bridges_lat, bridges_lon, village_boundaries):

    # Check CRS of village boundaries gdf
    print(f'The CRS of the village boundaries gdf is: {village_boundaries.crs}')

    # Create lat/lon variables
    lon = bridges[bridges_lon]
    lat = bridges[bridges_lat]

    # Create gdf of bridges data by converting lat/lon values to list of Shapely Point objects
    bridge_points = gpd.GeoDataFrame(bridges, geometry=gpd.points_from_xy(x=lon, y=lat), crs='EPSG:4326')

    # Set CRS of bridges gdf to CRS of village boundaries gdf
    bridge_points.to_crs(village_boundaries.crs, inplace=True)

    # Check that reprojection was successful
    print(f'The CRS of the bridges gdf is: {bridge_points.crs}')
    
    return bridge_points

bridge_points = map_bridges(bridges = bridge_sites,
                            bridges_lat = "Bridge Opportunity: GPS (Latitude)",
                            bridges_lon = "Bridge Opportunity: GPS (Longitude)",
                            village_boundaries = Rwanda_village_boundaries)

The CRS of the village boundaries gdf is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
The CRS of the bridges gdf is: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_

In [7]:
# WorldPop raster
# Note: This was pre-projected to the Rwanda TM CRS

# Specify file path for Worldpop raster
worldpop_fp = os.path.join(unsynced_path, "reprojected_population_raster.tif")

# Print CRS of raster
with rasterio.open(worldpop_fp) as src:
    print(src.crs)

PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Task 2: Create 1 km, 1.5 km, and 2 km buffers around each bridge site

In [8]:
def create_bridge_buffers(bridges, buffer_distance):

    # Create buffers around bridges according to user's distance input
    buf = bridges.geometry.buffer(distance=buffer_distance)

    # Add bridges point attribute information to buffers
    bridge_buffers = bridge_points.copy()
    bridge_buffers['geometry'] = buf

    return bridge_buffers

bridge_buffers_1km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 1000)
bridge_buffers_1_5km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 1500)
bridge_buffers_2km = create_bridge_buffers(bridges = bridge_points, buffer_distance = 2000)

In [9]:
# Create a base map
buffer_map = folium.Map(location=[-2.0, 30.0], zoom_start=8)

# Map bridge points and buffers
GeoJson(bridge_buffers_1km, name="1 km Buffers").add_to(buffer_map)
GeoJson(bridge_buffers_1_5km, name="1.5 km Buffers").add_to(buffer_map)
GeoJson(bridge_buffers_2km, name="2 km Buffers").add_to(buffer_map)

# Add layer control to toggle between layers
folium.LayerControl().add_to(buffer_map)

# Display the map
#buffer_map

Task 3: Extract population estimates from WorldPop raster for each buffer

In [7]:
def extract_pop(reprojected_pop_raster, population_raster_name, villages_near_bridges):
    
    # Copy input geodataframe to avoid modifying the original
    villages_copy = villages_near_bridges.copy()
    
    with rasterio.open(reprojected_pop_raster) as src:
        # Print information from raster profile
        print("CRS:", src.crs)
        print("Raster shape:", src.shape)
        print("Number of Bands:", src.count)

        # Iterate over each village and extract population data
        results = []
        for idx, row in villages_copy.iterrows():
            try:
                out_image = mask(src, [row.geometry], crop=True)
                out_image = out_image[0]

                # Handle masked arrays and replace with 0
                if np.ma.is_masked(out_image):
                    out_image = out_image.filled(0)

                # Remove negative values (e.g., nodata placeholders)
                out_image = np.where(out_image < 0, 0, out_image)

                total_population = np.sum(out_image)

            except Exception as e:
                print(f"Error at village {idx}: {e}")
                total_population = 0  # or np.nan

            results.append(total_population)

        # Add population data to the gdf
        villages_copy[population_raster_name] = results

    return villages_copy

pop_estimates_1km = extract_pop(reprojected_pop_raster = worldpop_fp,
                                population_raster_name = "WorldPop_1km",
                                villages_near_bridges = bridge_buffers_1km)

pop_estimates_1_5km = extract_pop(reprojected_pop_raster = worldpop_fp,
                                  population_raster_name = "WorldPop_1_5km",
                                  villages_near_bridges = bridge_buffers_1_5km)

pop_estimates_2km = extract_pop(reprojected_pop_raster = worldpop_fp,
                                population_raster_name = "WorldPop_2km",
                                villages_near_bridges = bridge_buffers_2km)

CRS: PROJCS["TM_Rwanda",GEOGCS["ITRF2005",DATUM["International_Terrestrial_Reference_Frame_2005",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6896"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",30],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",5000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Raster shape: (2145, 2455)
Number of Bands: 1
Error at village 173: Input shapes do not overlap raster.
Error at village 197: Input shapes do not overlap raster.
Error at village 198: Input shapes do not overlap raster.
Error at village 459: tuple index out of range
Error at village 542: tuple index out of range
Error at village 548: Input shapes do not overlap raster.
Error at village 596: Input shapes do not overlap raster.
Error a

In [8]:
# Combine population estimates into a single geodataframe
pop_estimates_combined = bridge_points.copy()

# Add population estimates from each buffer to the combined geodataframe
# Note: The merge is done on the index, which is the same for all geodataframes
pop_estimates_combined = pop_estimates_combined.merge(
    pop_estimates_1km[['WorldPop_1km']], left_index=True, right_index=True
).merge(
    pop_estimates_1_5km[['WorldPop_1_5km']], left_index=True, right_index=True
).merge(
    pop_estimates_2km[['WorldPop_2km']], left_index=True, right_index=True
)

In [ ]:
# Update the population estimates to replace 0 with NaN
# Note: The zeroes are because a number of villages are outside of Rwanda (the raster extent)
pop_estimates_combined['WorldPop_1km'] = pop_estimates_combined['WorldPop_1km'].replace(0, np.nan)

Task 4: Calculate summary statistics

In [12]:
# Calculate mean population for each distance radius
def mean_pop(pop_estimates_combined, pop_column):
    mean_value = pop_estimates_combined[pop_column].mean()
    print(f"The mean of {pop_column} is: {mean_value}")
    return mean_value

mean_pop(pop_estimates_combined, 'WorldPop_1km')
mean_pop(pop_estimates_combined, 'WorldPop_1_5km')
mean_pop(pop_estimates_combined, 'WorldPop_2km')
mean_pop(pop_estimates_combined, "Bridge Opportunity: Population Estimate 2000m")

The mean of WorldPop_1km is: 2912.153118543557
The mean of WorldPop_1_5km is: 6404.318211353873
The mean of WorldPop_2km is: 11152.916791902302
The mean of Bridge Opportunity: Population Estimate 2000m is: 9809.981109123435


np.float64(9809.981109123435)

In [13]:
# Calculate mean population for sites with value "Rwanda Full Scale Research - Randomized but Not Baselined"
# OR "Rwanda Full Scale Research - Randomized and Baselined - Long Term Control" in the Research Initiative field
print("Mean population for sites with research initiatives:")

# Define research values of interest
research_values = ["Rwanda Full Scale Research - Randomized but Not Baselined",
                   "Rwanda Full Scale Research - Randomized and Baselined - Long Term Control"]

# Calculate mean population for the defined research values
mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_1km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_1_5km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "WorldPop_2km"
)

mean_pop(
    pop_estimates_combined[
        pop_estimates_combined['Bridge Opportunity: Research Initiative'].isin(research_values)
    ], "Bridge Opportunity: Population Estimate 2000m"
)

Mean population for sites with research initiatives:
The mean of WorldPop_1km is: 2517.9408670127937
The mean of WorldPop_1_5km is: 5631.875188145069
The mean of WorldPop_2km is: 9768.744812571675
The mean of Bridge Opportunity: Population Estimate 2000m is: 7091.874036697248


np.float64(7091.874036697248)